In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Set HuggingFace cache directory to scratch to save space.
import os
os.environ['HUGGINGFACE_HUB_CACHE'] = '/scratch/' + os.environ['USER'] + '/huggingface_cache'

In [20]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "./llama-huggingface/llama-2-7b-chat", torch_dtype=torch.bfloat16
).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(
    "./llama-huggingface/llama-2-7b-chat"
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
import pickle

with open("wikitext103-v1-filtered.pkl", "rb") as f:
    texts = pickle.load(f)

In [5]:
add_padding = False

if add_padding:
    # https://discuss.huggingface.co/t/llama2-pad-token-for-batched-inference/48020
    # add_special_tokens doesn't work for some reason
    # tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    tokenizer.pad_token = "[PAD]"
    tokenizer.padding_side = "left"
    
    # Mini-test
    def mini_test():
        tokenization = tokenizer(texts[:1], padding=True, return_attention_mask=True, return_tensors='pt')
        sequence_lengths = tokenization.attention_mask.sum(dim=-1)
        print(sequence_lengths)
        # print(tokenization.input_ids[0, -sequence_lengths[0]:])
        # print(tokenizer.decode(tokenization.input_ids[0]))
        # assert (tokenization.input_ids[0, -sequence_lengths[0]:] > 0).all()
    
    mini_test()

In [21]:
# create a dictionary of models / optimizers

import autoencoder

cfg = {
    "act_size": 4096,
    "dict_size": 4096 * 128,
    "enc_dtype": "bf16",
    "l1_coeff": 3e-3,
    "seed": 0,
    "device": "cuda"
}

# might eventually have one model per layer
encoder = autoencoder.AutoEncoder(cfg=cfg)
encoder_optim = torch.optim.Adam(encoder.parameters())


In [24]:
import tqdm
import wandb

def run():
    wandb.init(project='llm-mechanics', config={'expansion_factor': 128})
    
    index = 0
    # batch_size = 1
    chunk_id = 0
    counter = 0
    with tqdm.tqdm(desc='Running inference', total=len(texts)) as pbar:
        while index < len(texts):
            # batch = texts[index:index + batch_size]
            tokenization = tokenizer(
                texts[index],
                return_tensors='pt',
                return_attention_mask=True,
                padding=False,
            ).to('cuda')
            sequence_lengths = tokenization.attention_mask.sum(dim=-1)
            with torch.no_grad():
                outputs = model.forward(
                    **tokenization,
                    output_hidden_states=True,
                )
                # (1 + n_layers) tuple of tensors [batch_size, sequence_length, d_model=4096]
                hidden_states = outputs.hidden_states

            # calculate reconstruction loss for autoencoder and train weights
            acts = torch.stack(hidden_states, dim=0).view(-1, 4096)
            loss, x_reconstruct, mid_acts, l2_loss, l1_loss = encoder(acts)
            loss.backward()
            wandb.log({'l2_loss': l2_loss.item(), 'l1_loss': l1_loss.item(), 'loss': loss.item()})
            encoder.make_decoder_weights_and_grad_unit_norm()
            encoder_optim.step()
            encoder_optim.zero_grad()

            # if index % 256 == 0 and index > 0:
            #     # Store the chunk.
            #     torch.save(hidden_states_chunk, f"hidden_states_{chunk_id}.pt")
            #     hidden_states_chunk.clear()
            #     chunk_id += 1

            if index % 1024 == 0 and index > 0:
                torch.cuda.empty_cache()

            index += 1
            pbar.update()

run()


wandb: Currently logged in as: myfatemi. Use `wandb login --relogin` to force relogin


Running inference:   1%|          | 5866/749962 [12:56<27:22:04,  7.55it/s] 


KeyboardInterrupt: 